In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium as fm
import numpy as np
from math import radians, cos, sin, asin, sqrt
from copy import deepcopy 
from scipy.stats import norm, kurtosis
import random

In [2]:
df = pd.read_csv('sites.csv') #read the data from the csv as a pandas dataframe
dists = np.load('dists.npy')
data = [df,dists]

def Fair(distribution, agegroups):
    #adapted the fairness formula to take into account that kids/adults/seniors
    #get different amounts of food by design
    allocations = {0:70, 1:56, 2:42} #0 is kids, 1 is adults, 2 is seniors. 
    if len(distribution) == 0:
        return 0
    
    totalsupply = sum(distribution)
    
    fairscore = 0
    for i in range(len(distribution)):
        recieved = distribution[i]/totalsupply
        allocated = allocations[agegroups[i]]/totalsupply
        fairscore += (recieved-allocated)**2
    
    fairscore = 1/(len(distribution)*fairscore+1)

    return fairscore

def returndata(data):
    df = data[0]
    dists = data[1]
    
    stdevs = np.array((df['StDev(Demand per Visit)']))
    variance = stdevs**2
    stdevs = np.sqrt(np.matmul(variance,dists))
    demands = np.array((df['Average Demand per Visit']))
    increment = np.matmul(demands/(70/772*365), dists) 
    
    return demands, stdevs, increment

def runsite(site, data, RANDOM_DEMAND = True):
    demands, stdevs, na = returndata(data) #gives the data we need    
    expdemand = demands[site]#expected demand for this site
    stdev = stdevs[site]     #standard deviation of demand at this site
    if RANDOM_DEMAND:
        demand = int(random.normalvariate(expdemand,stdev))#normally distributed random variable   
    else:
        demand = expdemand
    
    daysoffood = 14 #how many days we expect the food to last everyone
    ages = np.array([.41,.4,.19]) #proportions of kids/adults/seniors
    fooddist = np.array([5,4,3]) #how many lbs of food kids/ad/sen eat each day
    evfood = np.dot(ages,fooddist*daysoffood) #expected value of food demanded by random person
    
    totalfood = min(15000,(expdemand+1.645*stdev)*evfood)
    #bring enough food so that you have >50 pounds per person 95% of the time
    
    ppl_food_quantity = []      # initialize array of X_i's for ppl's quanitity in lbs for 2 weeks
    agegroups = [] #age breakdown of crowd
    
    for i in range(0,demand):  # for each person at a site
        expleft = max(int(expdemand*(1-(i/demand))),1) #expected remaining people
        foodleft = totalfood-sum(ppl_food_quantity) #remaining food
        expfoodperperson = foodleft/expleft #expected remaining food/person

        ratio = 1 #default is don't change anything        
        if expfoodperperson > evfood+10: #if we have a lot more food than we expect
            ratio = expfoodperperson/evfood #ratio>1
        elif expfoodperperson < evfood-10: #if we have a lot less food than we expect
            ratio = expfoodperperson/evfood #ratio < 1

        x = random.random()#random number to decide age of person
        if x<=ages[0]: #record their age group
                agegroups.append(0)
        elif x<=ages[0]+ages[1]:
            agegroups.append(1)
        else:
            agegroups.append(2)

        if sum(ppl_food_quantity)>totalfood: #if were out of food
            ppl_food_quantity.append(0) #you dont get any
        else:
            if x<=ages[0]:     # the person is a child
                ppl_food_quantity.append(min(100,max(35,fooddist[0]*daysoffood*ratio)))         # 5 lbs of food per day
                agegroups.append(0)
            elif x <= ages[0]+ages[1]:       # the person is an adult
                ppl_food_quantity.append(min(125,max(28,fooddist[1]*daysoffood*ratio)))         # 4lbs of food a day
                agegroups.append(1)
            else:                    # the person is a senior
                ppl_food_quantity.append(min(80,max(21,fooddist[2]*daysoffood*ratio)))       # 3lbs of food a day
                agegroups.append(2)
    # quantitative meansurement of fairness
    fairness_val = Fair(ppl_food_quantity, agegroups)

    if 0 not in ppl_food_quantity: #
        max_ppl_srvd = demand
    else:
        max_ppl_srvd = ppl_food_quantity.index(0)
        
    return fairness_val, max_ppl_srvd, sum(ppl_food_quantity)

In [3]:
#USE THIS CELL TO GENERATE THE SCHEDULES AND VALUES FOR THE PARETO FRONT
data = [df,dists]
demands,stdevs,increment = returndata(data)

schedules = [] 
demandDist = []
fairnesslevels = []

nvisits = np.zeros(70)

factorincrements = 50
maxfactor = 5
inc = maxfactor/factorincrements
factor = 0

while factor <= maxfactor:
    print(factor)
    schedule = [] #empty list for schedule
    demands = np.array((df['Average Demand per Visit']))
    daysElapsed = np.zeros(70)
    excessdemand = 0
    nsrvd = 0
    fairness = 0
    
    for day in range(365):
        
        fx = daysElapsed*factor
        demands = demands + fx

        a = deepcopy(demands)
        a = np.sort(a)

        first = a[-1]
        index1 = np.where(demands == first)
        second = a[-2]
        index2 = np.where(demands == second)

        demands = demands - fx
        
        fair1, maxsrvd1, pds1 = runsite(index1, data, False)
        fair2, maxsrvd2, pds2 = runsite(index2, data, False)
        
        fairness += fair1*maxsrvd1 + fair2*maxsrvd2
        nsrvd += maxsrvd1 + maxsrvd2
        
        demands = (demands - min(254,pds1/59.08)*dists[index1])[0]
        demands = (demands - min(254,pds1/59.08)*dists[index2])[0]
        
        daysElapsed = daysElapsed+1
        daysElapsed[int(index1[0])] = 0
        daysElapsed[int(index2[0])] = 0

        schedule.append([int(index1[0]),int(index2[0])])
        nvisits[index1] += 1
        nvisits[index2] += 1

        demands = [0 if x<0 else x for x in demands]
        demands = demands + increment
        excessdemand += sum(demands)

    schedules.append(schedule)
    fairnesslevels.append(float(fairness/nsrvd))
    demandDist.append(excessdemand/365)
    
    factor += inc

print(demandDist)
print(fairnesslevels)

0


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
freqList = []
for level in schedules:
    freq = [0] * 70
    for day in level:
        freq[day[0]] += 1
        freq[day[1]] += 1
    freqList.append(freq)

In [ ]:
def gini(list):
    ## first sort
    arr = np.array(list)
    sorted_arr = arr.copy()
    sorted_arr.sort()
    n = arr.size
    coef_ = 2. / n
    const_ = (n + 1.) / n
    weighted_sum = sum([(i+1)*yi for i, yi in enumerate(sorted_arr)])
    return coef_*weighted_sum/(sorted_arr.sum()) - const_

def identify_pareto(scores):
    # Count number of items
    population_size = scores.shape[0]
    # Create a NumPy index for scores on the pareto front (zero indexed)
    population_ids = np.arange(population_size)
    # Create a starting list of items on the Pareto front
    # All items start off as being labelled as on the Parteo front
    pareto_front = np.ones(population_size, dtype=bool)
    # Loop through each item. This will then be compared with all other items
    for i in range(population_size):
        # Loop through all other items
        for j in range(population_size):
            # Check if our 'i' pint is dominated by our 'j' point
            if all(scores[j] < scores[i]) and any(scores[j] <= scores[i]):
                # j dominates i. Label 'i' point as not on Pareto front
                pareto_front[i] = 0
                # Stop further comparisons with 'i' (no more comparisons needed)
                break
    # Return ids of scenarios on pareto front
    return population_ids[pareto_front]

def lorenz_curve(freq):
    X = np.array(freq)
    X_lorenz = X.cumsum() / X.sum()
    X_lorenz = np.insert(X_lorenz, 0, 0) 
    X_lorenz[0], X_lorenz[-1]
    fig, ax = plt.subplots(figsize=[6,6])
    ## scatter plot of Lorenz curve
    ax.scatter(np.arange(X_lorenz.size)/(X_lorenz.size-1), X_lorenz, 
               marker='x', color='darkgreen', s=100)
    ## line plot of equality
    ax.plot([0,1], [0,1], color='k')

In [ ]:
freqList[0].sort()

In [ ]:
lorenz_curve(freqList[0])

In [ ]:
unfairness = [1 - i for i in fairnesslevels] #minimize unfairness = maximize fairness
giniList = []
for i in freqList:
    giniList.append(gini(i))
minimize = np.column_stack((demandDist, giniList, unfairness))

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
pareto = identify_pareto(minimize)
pareto_front = minimize[pareto]
x_all = minimize[:, 0]
y_all = minimize[:, 1]
z_all = minimize[:, 2]
x_pareto = pareto_front[:, 0]
y_pareto = pareto_front[:, 1]
z_pareto = pareto_front[:, 2]


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_all, y_all, z_all)
ax.plot(x_pareto, y_pareto, z_pareto, color='r')
ax.set_xlabel('Demand')
ax.set_ylabel('Gini Coefficient')
ax.set_zlabel('Unfairness Index')
plt.show()

In [ ]:
unfairnesslevels = [1-x for x in fairnesslevels]

scaleddemand = (demandDist-min(demandDist))/(max(demandDist)-min(demandDist))
scaledfairness = (max(fairness) - fairness)/(max(fairness)-min(fairness))
scaledgini = (giniList-min(giniList))/(max(giniList)-min(giniList))

weightedarray = .5*scaleddemand+.3*scaledfairness+.2*scaledgini
ideal = np.where(weightedarray == min(weightedarray))[0][0]
print(ideal)

bestschedule = schedules[ideal][0]
#print(bestschedule) #our proposed 365 day schedule
totaldemand = demandDist[ideal]
ourfairness = fairnesslevels[ideal]
ginicoeff = giniList[ideal]

print(totaldemand,ourfairness,ginicoeff)


In [ ]:
def con_max(priorities, demand, leftover):
    index = -1
    for i in range(len(priorities)):
        if priorities[i] > priorities[index] and demand[i] <= leftover:
            index = i
    return index

In [ ]:
countList = []
for year in schedules:
    freq = [0] * 70
    for day in year:
        freq[day[0]] += 1
        freq[day[1]] += 1
    countList.append(freq)

In [ ]:
def locate_min(a):
    smallest = min(a)
    return [index for index, element in enumerate(a) if smallest == element]
def locate_max(a):
    biggest = max(a)
    return [index for index, element in enumerate(a) if biggest == element]

In [ ]:
nSchedule = []
avgDemands = df['Average Demand per Visit']
for i in schedules[9][0]:
    daily = []
    for j in i:
        leftover = (15000/60 - (avgDemands[j]))
        if leftover >= min(avgDemands):
            rare = locate_min(countList[9])
            rare_demand = []
            for i in rare:
                rare_demand.append(avgDemands[i])
            second = locate_max(rare_demand)
            visit = avgDemands[avgDemands == rare_demand[second[0]]]
            index = int(visit.index.tolist()[0])
            daily.append([j, index])
            countList[9][index] += 1
        else:
            daily.append([j, -1])
    nSchedule.append(daily)
        
nSchedule

In [ ]:
foodMap = fm.Map(location=[42.3, -76.5], tiles='Stamen Toner', zoom_start = 9.2)
coordinates = df[['latitude', 'longitude']]
coordinateList = coordinates.values.tolist()
print(coordinateList)
for element in coordinateList:
    fm.CircleMarker(element, radius=3, color='blue', opacity=0.3).add_to(foodMap)
foodMap

In [ ]:
import numpy as np
import random


def FairPriorities(distribution, agegroups):
    #adapted the fairness formula to take into account that kids/adults/seniors
    #get different amounts of food by design
    allocations = {0:70, 1:56, 2:42} #0 is kids, 1 is adults, 2 is seniors. 
    if len(distribution) == 0:
        return 0
    
    totalsupply = sum(distribution)
    
    fairscore = 0
    for i in range(len(distribution)):
        recieved = distribution[i]/totalsupply
        allocated = allocations[agegroups[i]]/totalsupply
        fairscore += (recieved-allocated)**2
    
    fairscore = 1/(len(distribution)*fairscore+1)

    return fairscore

def returndata(data):
    df = data[0]
    dists = data[1]
    
    stdevs = np.array((df['StDev(Demand per Visit)']))
    variance = stdevs**2
    stdevs = np.sqrt(np.matmul(variance,dists))
    demands = np.array((df['Average Demand per Visit']))
    increment = np.matmul(demands/(70/772*365), dists) 
    
    return demands, stdevs, increment

def runsite(site, data): #site is integer representing the visited site
                        #data is [dataframe, dists]
    demands, stdevs, na = returndata(data) #gives the data we need    
    expdemand = demands[site]#expected demand for this site
    stdev = stdevs[site]#standard deviation of demand at this site
    #normally distributed random variable   
    demand = int(random.normalvariate(expdemand,stdev))  
    
    daysoffood = 14 #how many days we expect the food to last everyone
    ages = np.array([.41,.4,.19]) #proportions of kids/adults/seniors
    fooddist = np.array([5,4,3]) #how many lbs of food kids/ad/sen eat each day
    evfood = np.dot(ages,fooddist*daysoffood) #expected value of food demanded by random person
    
    totalfood = min(15000,(expdemand+1.645*stdev)*evfood)
    #bring enough food so that you have >50 pounds per person 95% of the time
    #z-score of 1.645 corresponds to .95 probability
    
    ppl_food_quantity = []      # initialize array of X_i's for ppl's quanitity in lbs for 2 weeks
    agegroups = [] #age breakdown of crowd
    
    for i in range(0,demand):  # for each person at a site
        expleft = max(int(expdemand*(1-(i/demand))),1) #expected remaining people
        foodleft = totalfood-sum(ppl_food_quantity) #remaining food
        expfoodperperson = foodleft/expleft #expected remaining food/person
        
        ratio = 1 #default is don't change anything        
        if expfoodperperson > evfood+10: #if we have a lot more food than we expect
            ratio = expfoodperperson/evfood #ratio>1
        elif expfoodperperson < evfood-10: #if we have a lot less food than we expect
            ratio = expfoodperperson/evfood #ratio < 1
            
        x = random.random()#random number to decide age of person
        if x<=ages[0]: #record their age group
                agegroups.append(0)
        elif x<=ages[0]+ages[1]:
            agegroups.append(1)
        else:
            agegroups.append(2)
            
        if sum(ppl_food_quantity)>totalfood: #if were out of food
            ppl_food_quantity.append(0) #you dont get any
        else:
            if x<=ages[0]:     # the person is a child
                ppl_food_quantity.append(min(100,max(35,fooddist[0]*daysoffood*ratio)))         # 5 lbs of food per day
                agegroups.append(0)
            elif x <= ages[0]+ages[1]:       # the person is an adult
                ppl_food_quantity.append(min(125,max(28,fooddist[1]*daysoffood*ratio)))         # 4lbs of food a day
                agegroups.append(1)
            else:                    # the person is a senior
                ppl_food_quantity.append(min(80,max(21,fooddist[2]*daysoffood*ratio)))       # 3lbs of food a day
                agegroups.append(2)
    # quantitative meansurement of fairness
    fairness_val = FairPriorities(ppl_food_quantity, agegroups)

    if 0 not in ppl_food_quantity: #
        max_ppl_srvd = demand
    else:
        max_ppl_srvd = ppl_food_quantity.index(0)
        
    return fairness_val, max_ppl_srvd, sum(ppl_food_quantity)

def gini(list):
    ## first sort
    arr = np.array(list)
    sorted_arr = arr.copy()
    sorted_arr.sort()
    n = arr.size
    coef_ = 2. / n
    const_ = (n + 1.) / n
    weighted_sum = sum([(i+1)*yi for i, yi in enumerate(sorted_arr)])
    return coef_*weighted_sum/(sorted_arr.sum()) - const_

In [ ]:
testschedule(schedule)

In [ ]:
def testschedule2(schedule):
    
    df = pd.read_csv('sites.csv') #read the data from the csv as a pandas dataframe
    dists = np.load('dists.npy')
    data = [df,dists]
    
    demands, stdevs, increment = returndata(data)
    ntrials = 2
    
    avgfairness = np.zeros(ntrials)
    avgdemand = np.zeros(ntrials)
    ginicoeff = np.zeros(ntrials)
    totalfood = np.zeros(ntrials)
    for i in range(ntrials):
        nsrvd = 0
        fairness = 0
        excessdemand = 0
        totallbs = 0
        nvisits = np.zeros(70)
        for day in schedule:    
            site1 = day[0][0]
            site12 = day[0][1]
            site2 = day[1][0]
            site22 = day[1][1]
            
            fair1, nsrvd1, lbs1 = runsite(site1, data)
            fair2, nsrvd2, lbs2 = runsite(site2, data)
            
            fairness += fair1*nsrvd1 + fair2*nsrvd2
            nsrvd += nsrvd1 + nsrvd2
            
            demands = (demands - min(254, lbs1/59.08)*dists[site1])
            demands = (demands - min(254, lbs2/59.08)*dists[site2])
            
            #print(demands)
            nvisits[site1] += 1
            nvisits[site2] += 1
            
            if site12 != -1:
                fair12, nsrvd12, lbs12 = runsite(site12, data)
                fairness += fair12*nsrvd12
                nsrvd += nsrvd12
                demands = (demands - min(254, lbs12/59.08)*dists[site12])
                nvisits[site12] += 1
                totallbs += lbs12
            
            if site22 != -1:
                fair22, nsrvd22, lbs22 = runsite(site22, data)
                fairness += fair22*nsrvd22
                nsrvd += nsrvd22
                demands = (demands - min(254, lbs22/59.08)*dists[site22])
                nvisits[site22] += 1
                totallbs += lbs22
        
            demands = [0 if x<0 else x for x in demands]
            demands = demands + increment
            excessdemand += sum(demands)
            totallbs += lbs1+lbs2
            
        nvisits.sort()
        
        avgdemand[i] = excessdemand/365
        avgfairness[i] = fairness/nsrvd
        ginicoeff[i] = gini(nvisits)
        totalfood[i] = totallbs
    
    Average_Demand = np.mean(avgdemand)
    Average_Fairness = np.mean(avgfairness)
    Total_Food = np.mean(totallbs)
    Gini_Coefficient = gini(nvisits)

    print('Average Demand: ', Average_Demand)
    print('Average Fairness: ', Average_Fairness)
    print('Total Food Distributed: ', Total_Food)
    print('Gini Coefficient: ', Gini_Coefficient)
    
    return [Average_Demand, Average_Fairness, Total_Food, Gini_Coefficient]

In [ ]:
testschedule2(nSchedule)

In [ ]:
nSchedule2 = []
avgDemands = df['Average Demand per Visit']
for i in schedules[9][0]:
    daily = []
    for j in i:
        leftover = (15000/60 - (avgDemands[j]))
        if leftover >= min(avgDemands):
            rare = locate_min(countList[9])
            rare_demand = []
            for i in rare:
                rare_demand.append(avgDemands[i])
            second = locate_max(rare_demand)
            visit = avgDemands[avgDemands == rare_demand[second[0]]]
            index = int(visit.index.tolist()[0])
            daily.append([j, index])
            countList[9][index] += 1
        else:
            daily.append([j, -1])
    nSchedule2.append(daily)
        
nSchedule2